# Assignment of cell nuclei morphological features to spots

In this notebook you'll find the function which attributes cell nuclei morphological features to spots.

The data used in this script are :
- visium data (ST)
- visiopharm data (cell nuclei morphological features)

# 1. Import packages and data

pip install scanpy

pip install opencv-python

In [1]:
import os
import pandas as pd
import cv2
from matplotlib import pyplot as plt

# for ST data
import scanpy as sc

# 2. Functions definition

In [10]:
def isInside(circle_x, circle_y, rad, x, y):
    """Find if x,y coordinates are in a circle

    Parameters
    ----------
    circle_x : string
        x circle
    circle_y : string
        y circle
    rad : int
        circle radius
    x : float
        x coordinate
    y : float
        y coordinate
        
    Returns
    -------
    True/False : bool
    """
    # Compare radius of circle with distance of its center from given point
    if ((x - circle_x) * (x - circle_x) +
        (y - circle_y) * (y - circle_y) <= rad * rad):
        return True;
    else:
        return False;

In [9]:
def merged_modularity(path_for_visium, path_tissue_positions, path_for_morphological_features, name):
    """Attribute cell nuclei morphological features to spots

    Parameters
    ----------
    path_for_visium : string
        Visium data path
    path_tissue_positions : string
        visium tissue position data path 
    path_for_morphological_features : string
        visiopharm data path
    name : string
        name of the sample
    """
    #load visium data         
    ST_visium = sc.read_visium(path_for_visium)
    ST_visium.obs = ST_visium.obs.reset_index()
    ST_visium.obs = ST_visium.obs.rename(columns={"index": "probes"})
    # load tissue positions df
    tissue_positions = pd.read_csv(path_tissue_positions, header = None)
    #preprocess tissue positions tissu df
    tissue_positions = tissue_positions.rename(columns={0: "probes"})
    #merge the two df
    coordinates_position_probes_visium = pd.merge(ST_visium.obs, tissue_positions, how='inner', on=['probes'])
    y_coordinates_position_probes_visium = list(coordinates_position_probes_visium[4])
    x_coordinates_position_probes_visium = list(coordinates_position_probes_visium[5])

    # create df form visium data
    ST_visium = sc.read_visium(path_for_visium)
    counts = pd.DataFrame(ST_visium.X.todense(), columns=ST_visium.var_names, index=ST_visium.obs_names) # create counts df

    # attribution of center nulcei coordinates in spot
    PDAC_B2 = pd.read_csv(path_for_morphological_features)
    PDAC_B2_visio = PDAC_B2.drop(columns = ["Unnamed: 0","Name","Total Nuclei (#)"])    
    PDAC_B2_visio_x = list(PDAC_B2["new_x"])
    PDAC_B2_visio_y = list(PDAC_B2["new_y"])
    rad = 65
    idx_to_drop = []
    x_ST_list = []
    y_ST_list = []
    for x,y in zip(PDAC_B2_visio_x, PDAC_B2_visio_y):
        for circle_x, circle_y in zip(x_coordinates_position_probes_visium, y_coordinates_position_probes_visium):
            if(isInside(circle_x, circle_y, rad, x, y)):
                msk = PDAC_B2_visio['new_x'].eq(x) & (PDAC_B2_visio['new_y'].eq(y))
                idx_to_drop.append(PDAC_B2_visio.index[msk])
                x_ST_list.append(circle_x)
                y_ST_list.append(circle_y)

    new_idx = [x for idx in idx_to_drop for x in idx]
    df_collect_morpho = PDAC_B2_visio.drop(new_idx)
    df_1notin2 = PDAC_B2_visio[~PDAC_B2_visio.isin(df_collect_morpho).all(axis=1)]

    df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
    df_1notin2["y_coord"] = y_ST_list
    df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per object', 'Texture inertia', 'Max Intensity',
                                    'Mean Intensity', 'Min Intensity', 'Modus', 'Texture Anisotropy',
                                    'Texture Entropy', 'Texture Major Axis', 'Texture Major Direction',
                                    'Texture Minor Axis'].mean() # calculate mean of morphological features by spot
    df_mean = df_mean.reset_index() # switch index to column

  

    coord = pd.DataFrame(ST_visium.obsm['spatial'], columns=['x_coord', 'y_coord'], index=ST_visium.obs_names) # create spot coordinates df
    merged_coord = pd.merge(coord, df_mean, on= ["x_coord", "y_coord"], how="left") # merged the two dataframe into new one
    coordinates_position_probes_visium = coordinates_position_probes_visium.rename(columns={4: "y_coord", 5: "x_coord"}) # rename columns
    merged_probes = pd.merge(coordinates_position_probes_visium, merged_coord, on= ["x_coord", "y_coord"], how="left") # merge coordinates_position_probes_visium and merged_coord df
    merged_probes_clean = merged_probes.drop(['in_tissue', 'array_row', 'array_col', 1,2,3], axis=1) # drop useless columns
    merged_probes_clean = merged_probes_clean.set_index('probes') # change index name
    final_df = counts.join(merged_probes_clean) # merged merged_probes_clean and counts df
    #save results
    final_df.to_csv(f'dataframe/{name}_profils_merged_morpho_raw.csv')
    counts_with_probes = final_df.drop(['Nuclei Area per object', 'Texture inertia',
       'Max Intensity', 'Mean Intensity', 'Min Intensity', 'Modus',
       'Texture Anisotropy', 'Texture Entropy', 'Texture Major Axis',
       'Texture Major Direction', 'Texture Minor Axis'], axis = 1)
    counts_with_probes.to_csv(f'dataframe/{name}_counts_with_probes.csv')                    
    print("Final df and count df correctly save !")
    print("Number of NA:")
    print(final_df.isna().sum()) 

# 3. DPLFC dataset

## DPLFC 151676

In [3]:
path_for_morphological_features = "preprocess_visiopharm/df_DPLFC_151676_without_background.csv"
path_for_visium = "./visium/151676/"
path_tissue_positions = "./visium/151676/spatial/tissue_positions_list.csv"
name = "DPLFC_151676"

In [6]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_77/3107094939.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_77/3107094939.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_77/3107094939.py:42: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per objec

Final df and count df correctly save !
Number of NA:
MIR1302-2HG                   0
FAM138A                       0
OR4F5                         0
AL627309.1                    0
AL627309.3                    0
                           ... 
Texture Anisotropy         1046
Texture Entropy            1046
Texture Major Axis         1046
Texture Major Direction    1046
Texture Minor Axis         1046
Length: 33551, dtype: int64


## DPLFC 151507

In [42]:
path_for_morphological_features = "preprocess_visiopharm/df_DPLFC_151507_without_background.csv"
path_for_visium = "./visium/151507/"
path_tissue_positions = "./visium/151507/spatial/tissue_positions_list.csv"
name = "DPLFC_151507"

In [43]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_77/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_77/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_77/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per objec

Final df and count df correctly save !
Number of NA:
MIR1302-2HG                  0
FAM138A                      0
OR4F5                        0
AL627309.1                   0
AL627309.3                   0
                          ... 
Texture Anisotropy         799
Texture Entropy            799
Texture Major Axis         799
Texture Major Direction    799
Texture Minor Axis         799
Length: 33551, dtype: int64


## DPLFC 151508

In [44]:
path_for_morphological_features = "preprocess_visiopharm/df_DPLFC_151508_without_background.csv"
path_for_visium = "./visium/151508/"
path_tissue_positions = "./visium/151508/spatial/tissue_positions_list.csv"
name = "DPLFC_151508"

In [45]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_77/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_77/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_77/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per objec

Final df and count df correctly save !
Number of NA:
MIR1302-2HG                  0
FAM138A                      0
OR4F5                        0
AL627309.1                   0
AL627309.3                   0
                          ... 
Texture Anisotropy         881
Texture Entropy            881
Texture Major Axis         881
Texture Major Direction    881
Texture Minor Axis         881
Length: 33551, dtype: int64


## DPLFC 151509

In [46]:
path_for_morphological_features = "preprocess_visiopharm/df_DPLFC_151509_without_background.csv"
path_for_visium = "./visium/151509/"
path_tissue_positions = "./visium/151509/spatial/tissue_positions_list.csv"
name = "DPLFC_151509"

In [47]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_77/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_77/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_77/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per objec

Final df and count df correctly save !
Number of NA:
MIR1302-2HG                  0
FAM138A                      0
OR4F5                        0
AL627309.1                   0
AL627309.3                   0
                          ... 
Texture Anisotropy         436
Texture Entropy            436
Texture Major Axis         436
Texture Major Direction    436
Texture Minor Axis         436
Length: 33551, dtype: int64


## DPLFC 151510

In [22]:
path_for_morphological_features = "preprocess_visiopharm/df_DPLFC_151510_without_background.csv"
path_for_visium = "./visium/151510/"
path_tissue_positions = "./visium/151510/spatial/tissue_positions_list.csv"
name = "DPLFC_151510"

In [23]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_618/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_618/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_618/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per ob

Final df and count df correctly save !
Number of NA:
MIR1302-2HG                  0
FAM138A                      0
OR4F5                        0
AL627309.1                   0
AL627309.3                   0
                          ... 
Texture Anisotropy         934
Texture Entropy            934
Texture Major Axis         934
Texture Major Direction    934
Texture Minor Axis         934
Length: 33551, dtype: int64


## DPLFC 151669

In [48]:
path_for_morphological_features = "preprocess_visiopharm/df_DPLFC_151669_without_background.csv"
path_for_visium = "./visium/151669/"
path_tissue_positions = "./visium/151669/spatial/tissue_positions_list.csv"
name = "DPLFC_151669"

In [49]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_77/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_77/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_77/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per objec

Final df and count df correctly save !
Number of NA:
MIR1302-2HG                  0
FAM138A                      0
OR4F5                        0
AL627309.1                   0
AL627309.3                   0
                          ... 
Texture Anisotropy         305
Texture Entropy            305
Texture Major Axis         305
Texture Major Direction    305
Texture Minor Axis         305
Length: 33551, dtype: int64


## DPLFC 151670

In [50]:
path_for_morphological_features = "preprocess_visiopharm/df_DPLFC_151670_without_background.csv"
path_for_visium = "./visium/151670/"
path_tissue_positions = "./visium/151670/spatial/tissue_positions_list.csv"
name = "DPLFC_151670"

In [51]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_77/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_77/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_77/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per objec

Final df and count df correctly save !
Number of NA:
MIR1302-2HG                  0
FAM138A                      0
OR4F5                        0
AL627309.1                   0
AL627309.3                   0
                          ... 
Texture Anisotropy         774
Texture Entropy            774
Texture Major Axis         774
Texture Major Direction    774
Texture Minor Axis         774
Length: 33551, dtype: int64


## DPLFC 151671

In [25]:
path_for_morphological_features = "preprocess_visiopharm/df_DPLFC_151671_without_background.csv"
path_for_visium = "./visium/151671/"
path_tissue_positions = "./visium/151671/spatial/tissue_positions_list.csv"
name = "DPLFC_151671"

In [26]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_618/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_618/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_618/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per ob

Final df and count df correctly save !
Number of NA:
MIR1302-2HG                  0
FAM138A                      0
OR4F5                        0
AL627309.1                   0
AL627309.3                   0
                          ... 
Texture Anisotropy         396
Texture Entropy            396
Texture Major Axis         396
Texture Major Direction    396
Texture Minor Axis         396
Length: 33551, dtype: int64


## DPLFC 151672

In [18]:
path_for_morphological_features = "preprocess_visiopharm/df_DPLFC_151672_without_background.csv"
path_for_visium = "./visium/151672/"
path_tissue_positions = "./visium/151672/spatial/tissue_positions_list.csv"
name = "DPLFC_151672"

In [19]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_618/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_618/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_618/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per ob

Final df and count df correctly save !
Number of NA:
MIR1302-2HG                  0
FAM138A                      0
OR4F5                        0
AL627309.1                   0
AL627309.3                   0
                          ... 
Texture Anisotropy         491
Texture Entropy            491
Texture Major Axis         491
Texture Major Direction    491
Texture Minor Axis         491
Length: 33551, dtype: int64


## DPLFC 151673

In [52]:
path_for_morphological_features = "preprocess_visiopharm/df_DPLFC_151673_without_background.csv"
path_for_visium = "./visium/151673/"
path_tissue_positions = "./visium/151673/spatial/tissue_positions_list.csv"
name = "DPLFC_151673"

In [53]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_77/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_77/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_77/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per objec

Final df and count df correctly save !
Number of NA:
MIR1302-2HG                  0
FAM138A                      0
OR4F5                        0
AL627309.1                   0
AL627309.3                   0
                          ... 
Texture Anisotropy         163
Texture Entropy            163
Texture Major Axis         163
Texture Major Direction    163
Texture Minor Axis         163
Length: 33551, dtype: int64


## DPLFC 151674

In [54]:
path_for_morphological_features = "preprocess_visiopharm/df_DPLFC_151674_without_background.csv"
path_for_visium = "./visium/151674/"
path_tissue_positions = "./visium/151674/spatial/tissue_positions_list.csv"
name = "DPLFC_151674"

In [55]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_77/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_77/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_77/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per objec

Final df and count df correctly save !
Number of NA:
MIR1302-2HG                  0
FAM138A                      0
OR4F5                        0
AL627309.1                   0
AL627309.3                   0
                          ... 
Texture Anisotropy         789
Texture Entropy            789
Texture Major Axis         789
Texture Major Direction    789
Texture Minor Axis         789
Length: 33551, dtype: int64


## DPLFC 151675

In [20]:
path_for_morphological_features = "preprocess_visiopharm/df_DPLFC_151675_without_background.csv"
path_for_visium = "./visium/151675/"
path_tissue_positions = "./visium/151675/spatial/tissue_positions_list.csv"
name = "DPLFC_151675"

In [21]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_618/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_618/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_618/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per ob

Final df and count df correctly save !
Number of NA:
MIR1302-2HG                  0
FAM138A                      0
OR4F5                        0
AL627309.1                   0
AL627309.3                   0
                          ... 
Texture Anisotropy         899
Texture Entropy            899
Texture Major Axis         899
Texture Major Direction    899
Texture Minor Axis         899
Length: 33551, dtype: int64


# 4. PDAC dataset

## A1

In [9]:
path_for_morphological_features = "preprocess_visiopharm/df_A1_without_background.csv"
path_for_visium = "./visium/PDAC_A1/"
path_tissue_positions = "./visium/PDAC_A1/spatial/tissue_positions_list.csv"
name = "PDAC_A1"

In [10]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_76/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_76/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_76/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per objec

Final df and count df correctly save !
Number of NA:
SAMD11                       0
NOC2L                        0
KLHL17                       0
PLEKHN1                      0
PERM1                        0
                          ... 
Texture Anisotropy         251
Texture Entropy            251
Texture Major Axis         251
Texture Major Direction    251
Texture Minor Axis         251
Length: 17956, dtype: int64


## B2

In [11]:
path_for_morphological_features = "preprocess_visiopharm/df_B2_without_background.csv"
path_for_visium = "./visium/PDAC_B2/"
path_tissue_positions = "./visium/PDAC_B2/spatial/tissue_positions_list.csv"
name = "PDAC_B2"

In [12]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_76/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_76/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_76/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per objec

Final df and count df correctly save !
Number of NA:
SAMD11                       0
NOC2L                        0
KLHL17                       0
PLEKHN1                      0
PERM1                        0
                          ... 
Texture Anisotropy         112
Texture Entropy            112
Texture Major Axis         112
Texture Major Direction    112
Texture Minor Axis         112
Length: 17956, dtype: int64


## C3

In [21]:
path_for_morphological_features = "preprocess_visiopharm/df_C3_without_background.csv"
path_for_visium = "./visium/PanCN_C3/"
path_tissue_positions = "./visium/PanCN_C3/spatial/tissue_positions_list.csv"
name = "PanCN_C3"

In [22]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_76/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_76/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_76/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per objec

Final df and count df correctly save !
Number of NA:
SAMD11                      0
NOC2L                       0
KLHL17                      0
PLEKHN1                     0
PERM1                       0
                           ..
Texture Anisotropy         70
Texture Entropy            70
Texture Major Axis         70
Texture Major Direction    70
Texture Minor Axis         70
Length: 17956, dtype: int64


## D4

In [23]:
path_for_morphological_features = "preprocess_visiopharm/df_D4_without_background.csv"
path_for_visium = "./visium/PanCN_D4/"
path_tissue_positions = "./visium/PanCN_D4/spatial/tissue_positions_list.csv"
name = "PanCN_D4"

In [24]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/tmp/ipykernel_76/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_76/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_76/2124656122.py:41: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df_1notin2.groupby(['x_coord','y_coord'])['Nuclei Area per objec

Final df and count df correctly save !
Number of NA:
SAMD11                      0
NOC2L                       0
KLHL17                      0
PLEKHN1                     0
PERM1                       0
                           ..
Texture Anisotropy         46
Texture Entropy            46
Texture Major Axis         46
Texture Major Direction    46
Texture Minor Axis         46
Length: 17956, dtype: int64


# 5. Mouse brain dataset

In [4]:
path_for_morphological_features = "preprocess_visiopharm/df_V1_Mouse_Brain_Sagittal_Anterior_image_without_background.csv"
path_for_visium = "./visium/V1_Mouse_Brain_Sagittal_Anterior/"
path_tissue_positions = "./visium/V1_Mouse_Brain_Sagittal_Anterior/spatial/tissue_positions_list.csv"
name = "V1_Mouse_Brain_Sagittal_Anterior"

In [5]:
merged_modularity(path_for_visium, path_tissue_positions,path_for_morphological_features, name)

/opt/conda/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/tmp/ipykernel_775/2124656122.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["x_coord"] = x_ST_list # add x and y spots coordinates in columns
/tmp/ipykernel_775/2124656122.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1notin2["y_coord"] = y_ST_list
/tmp/ipykernel_775/2124656122.py

Final df and count df correctly save !
Number of NA:
Xkr4                         0
Gm1992                       0
Gm19938                      0
Gm37381                      0
Rp1                          0
                          ... 
Texture Anisotropy         149
Texture Entropy            149
Texture Major Axis         149
Texture Major Direction    149
Texture Minor Axis         149
Length: 32298, dtype: int64


# 6. Save in sb

# DPFLC dataset

## 151507

In [7]:
# Save results in sb
counts = pd.read_csv("dataframe/DPLFC_151507_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/DPLFC_151507_counts_with_probes.csv")

## 151508

In [8]:
# Save results in sb
counts = pd.read_csv("dataframe/DPLFC_151508_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/DPLFC_151508_counts_with_probes.csv")

## 151509

In [9]:
# Save results in sb
counts = pd.read_csv("dataframe/DPLFC_151509_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/DPLFC_151509_counts_with_probes.csv")

## 151510

In [24]:
# Save results in sb
counts = pd.read_csv("dataframe/DPLFC_151510_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/DPLFC_151510_counts_with_probes.csv")

## 151669

In [10]:
# Save results in sb
counts = pd.read_csv("dataframe/DPLFC_151669_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/DPLFC_151669_counts_with_probes.csv")

## 151670

In [11]:
# Save results in sb
counts = pd.read_csv("dataframe/DPLFC_151670_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/DPLFC_151670_counts_with_probes.csv")

## 151671

In [27]:
# Save results in sb
counts = pd.read_csv("dataframe/DPLFC_151671_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/DPLFC_151671_counts_with_probes.csv")

## 151672

In [28]:
# Save results in sb
counts = pd.read_csv("dataframe/DPLFC_151672_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/DPLFC_151672_counts_with_probes.csv")

## 151673

In [12]:
# Save results in sb
counts = pd.read_csv("dataframe/DPLFC_151673_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/DPLFC_151673_counts_with_probes.csv")

## 151674

In [13]:
# Save results in sb
counts = pd.read_csv("dataframe/DPLFC_151674_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/DPLFC_151674_counts_with_probes.csv")

## 151675

In [29]:
# Save results in sb
counts = pd.read_csv("dataframe/DPLFC_151675_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/DPLFC_151675_counts_with_probes.csv")

## 151676

In [14]:
# Save results in sb
counts = pd.read_csv("dataframe/DPLFC_151676_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/DPLFC_151676_counts_with_probes.csv")

# PDAC dataset

## A1

In [28]:
# Save results in sb
counts = pd.read_csv("dataframe/PDAC_A1_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/A1_counts_with_probes.csv")

## B2

In [29]:
# Save results in sb
counts = pd.read_csv("dataframe/PDAC_B2_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/B2_counts_with_probes.csv")

## C3

In [30]:
# Save results in sb
counts = pd.read_csv("dataframe/PanCN_C3_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/C3_counts_with_probes.csv")

## D4

In [31]:
# Save results in sb
counts = pd.read_csv("dataframe/PanCN_D4_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/D4_counts_with_probes.csv")

# Mouse brain dataset

In [6]:
# Save results in sb
counts = pd.read_csv("dataframe/V1_Mouse_Brain_Sagittal_Anterior_counts_with_probes.csv")
# save in output files
sevenbg_output_path = f"/sbgenomics/output-files/data/counts_with_probes"
os.makedirs(sevenbg_output_path, exist_ok=True)
counts.to_csv(f"{sevenbg_output_path}/V1_Mouse_Brain_Sagittal_Anterior_counts_with_probes.csv")